## Training the feature extraction 

In [46]:
import pandas as pd

In [47]:
df = pd.read_csv('../data/mbti_full_pull_half_train.csv', index_col=0)

In [48]:
from Features import Features

cleaning the copora


KeyboardInterrupt: 

# Training the first layers

In [ ]:
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import datetime

## enumerating all the cognitive functions (With repeats)


In [ ]:
from itertools import combinations

types = ['IE','NS','TP','JF']

deg1 = []

for i in types:
    for ii in i:
        deg1.append(ii)

deg2 = []
for i,j, in combinations(types,2):
    for ii in i:
        for jj in j:
            deg2.append(ii+jj)  

deg3 = []
for i,j,k in combinations(types,3):
    for ii in i:
        for jj in j:
            for kk in k:
                deg3.append(ii+jj+kk)

deg4 = []
for i,j,k,l in combinations(types,4):
    for ii in i:
        for jj in j:
            for kk in k:
                for ll in l:
                    deg4.append(ii+jj+kk+ll)

cog_funs = deg1 + deg2 + deg3 + deg4

def normalize(s):
    ret = ''
    for type in types:
        if type[0] in s:
            ret += type[0]
        elif type[1] in s:
            ret += type[1]
        else:
            ret += '_'
    return ret

cog_funs = list(map(normalize,cog_funs))
print('\t'.join(cog_funs))
print(len(cog_funs))

I___	E___	_N__	_S__	__T_	__P_	___J	___F	IN__	IS__	EN__	ES__	I_T_	I_P_	E_T_	E_P_	I__J	I__F	E__J	E__F	_NT_	_NP_	_ST_	_SP_	_N_J	_N_F	_S_J	_S_F	__TJ	__TF	__PJ	__PF	INT_	INP_	IST_	ISP_	ENT_	ENP_	EST_	ESP_	IN_J	IN_F	IS_J	IS_F	EN_J	EN_F	ES_J	ES_F	I_TJ	I_TF	I_PJ	I_PF	E_TJ	E_TF	E_PJ	E_PF	_NTJ	_NTF	_NPJ	_NPF	_STJ	_STF	_SPJ	_SPF	INTJ	INTF	INPJ	INPF	ISTJ	ISTF	ISPJ	ISPF	ENTJ	ENTF	ENPJ	ENPF	ESTJ	ESTF	ESPJ	ESPF
80


There are repeating elements in the above listed cog_funs, such as 'I___' and 'E___' are really the same thing, and I choose not to handle this repeatition.  

Now, what's left to do is to have feature extractions and then train binary classifier for each cognitive functions. 

Suppose the feature extraction is trained and stored in a ../models/features______.model

In [ ]:
feature_extractor = '../models/features'+ "2021-12-11" + '.model'

with open(feature_extractor,'rb') as f:
    feature_extractor = pickle.load(f)

In [ ]:
train_csv_path = None
train_df = pd.read_csv('../data/mbti_full_pull_half_train.csv',index_col=0)
# this should have 2 cols. one is 'body' and another is 'label'

train_X = feature_extractor.get_features(train_df['body'])
train_y = train_df['mbti_type']

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/baihaocheng/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
def check_match(y,y_):
    for i,j in enumerate(y_):
        if j == '_':
            pass
        elif j == y[i]:
            pass
        else:
            return 0
    return 1

In [ ]:
check_match('INTJ','_NT_'), check_match('INTJ','E___')

(1, 0)

In [ ]:
modelNameSuffix = str(datetime.date.today())

## Training the first layer model

In [ ]:
for model in cog_funs:
    
    train_yy = [check_match(i,model) for i in train_y]
    
    classifier = RandomForestClassifier(n_estimators=20)
    # fill the parameter of this classifier such as max_depth, etc. Or perhaps use a better classifier? 
    
    classifier.fit(train_X, train_yy)
    
    with open('../models/first_layer/'  + model + modelNameSuffix + '.model','wb') as f:
        pickle.dump(classifier,f)
    print(model +' training completed')
    

# The second layer model

The inputs of the second layer model should be
- cognitive functions, there are roughly 80 of them. With a bigger weight
- the dimension-reducted output of the features. The features supposed have a really large dimension of 10k, we should perhaps reduct it down to 1k? 

And the layers of the NN should be
- a few linear layers, plus activations

I have already written some codes in `development.ipynb`. Those codes should be moved here. 